In [61]:
import cv2 as cv
from cv2 import VideoCapture
import numpy as np
from typing import Callable, Tuple

In [62]:
def is_bbox(array: np.ndarray) -> bool:
    if array is None:
        return False
    return array.ndim == 2 and array.shape[1] == 4 and array.dtype == int

In [63]:
def to_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

In [64]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image

    image = to_grayscale(image)

    image = cv.convertScaleAbs(image, alpha=1.3, beta=0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(image, 220, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=3)

    # cv.imshow("BBOXES", mask)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes

In [65]:
def resize_boxes(bboxes: np.ndarray, new_width: int, new_height: int) -> np.ndarray:
    # Validate bboxes arg
    assert is_bbox(bboxes)

    # Create width and heigh arrays
    num_boxes = bboxes.shape[0]
    new_width = np.full(shape=[num_boxes], fill_value=new_width)
    new_height = np.full(shape=[num_boxes], fill_value=new_height)

    # Unpack columns
    x, y, w, h = bboxes.T

    # Calc center of bbox
    x_mid = x + w // 2
    y_mid = y + h // 2

    # Calc upper left corner
    new_x = x_mid - new_width // 2
    new_y = y_mid - new_height // 2

    # Join the columns back
    new_bboxes = np.column_stack((new_x, new_y, new_width, new_height))
    return new_bboxes

In [66]:
def sanitize_boxes(bboxes: np.ndarray, image_shape: tuple[int]) -> np.ndarray:
    # Validate bboxes arg
    assert is_bbox(bboxes)

    # Get max possible dimensions
    max_height = image_shape[0] - 1
    max_width = image_shape[1] - 1

    # Unpack columns
    x, y, w, h = bboxes.T

    # Make sure bboxes have valid dimensions and within image shape
    good_mask = (x >= 0) & (y >= 0) & (w > 0) & (h > 0)
    good_mask = good_mask & (x + w <= max_width) & (y + h <= max_height)

    # Select only good bboxes
    bboxes = bboxes[good_mask]
    return bboxes

In [67]:
def remove_overlapping_boxes(bboxes: np.ndarray, bboxes_other: np.ndarray = None) -> np.ndarray:
    """
    If bboxes_other is not None then overlap is checked against these bboxes
    """
    # Validate args
    assert is_bbox(bboxes)
    assert bboxes_other is None or (is_bbox(bboxes_other) and bboxes.shape == bboxes_other.shape)

    # Get num elements
    num_boxes = bboxes.shape[0]

    # Calculate left, right, top, bottom limits
    left = np.expand_dims(bboxes[:, 0], axis=1)
    right = np.expand_dims(bboxes[:, 0] + bboxes[:, 2], axis=1)
    top = np.expand_dims(bboxes[:, 1], axis=1)
    bottom = np.expand_dims(bboxes[:, 1] + bboxes[:, 3], axis=1)

    # Calculate left, right, top, bottom limits of other
    if bboxes_other is None:
        left_other = left
        right_other = right
        top_other = top
        bottom_other = bottom
    else:
        left_other = np.expand_dims(bboxes_other[:, 0], axis=1)
        right_other = np.expand_dims(bboxes_other[:, 0] + bboxes_other[:, 2], axis=1)
        top_other = np.expand_dims(bboxes_other[:, 1], axis=1)
        bottom_other = np.expand_dims(bboxes_other[:, 1] + bboxes_other[:, 3], axis=1)

    # Check for left limit intrusions, right limit intrusions, ...
    check_l = (left <= left_other.T) & (left_other.T <= right)
    check_r = (left <= right_other.T) & (right_other.T <= right)
    check_t = (top <= top_other.T) & (top_other.T <= bottom)
    check_b = (top <= bottom_other.T) & (bottom_other.T <= bottom)

    # Check for combinations of left-top intrusions, left-bottom intrusions, ...
    check_lt = check_l & check_t
    check_lb = check_l & check_b
    check_rt = check_r & check_t
    check_rb = check_r & check_b

    # Get all combinations; get rid of self identical matches
    check = check_lt | check_lb | check_rt | check_rb
    check = np.bitwise_xor(check, np.eye(num_boxes, dtype=bool))
    check = np.argwhere(check)

    # Get unique indices of bad bboxes
    bad_indices = np.unique(check)

    # Get indices of good bboxes
    good_indices = np.arange(num_boxes)
    good_indices = good_indices[np.in1d(good_indices, bad_indices, invert=True)]

    # Take only the good bboxes
    good_bboxes = np.take(bboxes, good_indices, axis=0)
    return good_bboxes

In [68]:
def extract_slices(image: np.ndarray, bboxes: np.ndarray) -> list[np.ndarray]:
    # Validate args
    assert is_bbox(bboxes)

    slices = []
    for bbox in bboxes:
        # Deconstruct bbox
        x, y, w, h = bbox

        # Extract a slice and add to slice list
        slice = image[y : y + h, x : x + w]
        slices.append(slice)

    return slices

In [69]:
def extract_worms(
    image: np.ndarray,
) -> Tuple[np.ndarray, list[np.ndarray]]:
    # Apply transform if needed
    # Find bboxes according to given params
    bboxes = find_boxes(image)

    # Calc camera-sized bboxes
    camera_bboxes = resize_boxes(bboxes, new_width=400, new_height=400)

    # Calc worm-sized bboxes
    worm_bboxes = resize_boxes(camera_bboxes, 150, 150)

    # Remove overlapping bboxes between camera-bboxes and worm-bboxes
    camera_bboxes = remove_overlapping_boxes(camera_bboxes, worm_bboxes)

    # Remove bboxes which are out of bounds
    camera_bboxes = sanitize_boxes(camera_bboxes, image.shape)

    # Get corresponding image slices to the camera-bboxes
    image_slices = extract_slices(image, camera_bboxes)

    return camera_bboxes, image_slices

In [70]:
from pathlib import Path


def save_samples(image_list: list[np.ndarray], sample_name: str):
    """
    The naming scheme of the file should be: "filename{}.jpg"
    The braces will be replaced by the sample number.
    """
    folder = Path(sample_name).parent
    Path(folder).mkdir(parents=True, exist_ok=True)

    for i, img in enumerate(image_list):
        img_name = sample_name.format(i)
        cv.imwrite(img=img, filename=img_name)

In [71]:
cap = VideoCapture("worms.avi")

i = 0
while True:
    ret, image = cap.read()
    if ret == False:
        break

    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    coords, rois = extract_worms(image)

    # save_samples(rois, "rois/img{iter}_{{}}.jpg".format(iter=i))

    # Draw bboxes
    for box in coords:
        x, y, w, h = box
        cv.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 3)

    N = 64
    image = to_grayscale(image)
    image = (np.round(image * (N / 255), 0) * (255 / N)).astype(np.uint8)

    cv.imshow("BBOXES", image)

    i += 1

cap.release()
cv.destroyAllWindows()

In [72]:
cv.destroyAllWindows()